In [1]:
# Modules: gdal/3.6.4  
# Environment base: /g/data/xe2/John/geospatenv

In [2]:
# Note: For the canopy_height.download_new_tiles to work, you need to create an AWS account, then create a file named .aws/credentials in your home directory (e.g. /home/147/cb8590) with these contents:
# [default]
# aws_access_key_id = ACCESS_KEY
# aws_secret_access_key = SECRET_KEY

In [1]:
# Standard library
import os

# Local imports
os.chdir(os.path.join(os.path.expanduser('~'), "Projects/PaddockTS"))
from DAESIM_preprocess.util import *
from DAESIM_preprocess.terrain_tiles import terrain_tiles
from DAESIM_preprocess.slga_soils import slga_soils, asris_urls
from DAESIM_preprocess.ozwald_yearly import ozwald_yearly_average
from DAESIM_preprocess.ozwald_8day import ozwald_8day, ozwald_8day_abbreviations
from DAESIM_preprocess.ozwald_daily import ozwald_daily, ozwald_daily_abbreviations
from DAESIM_preprocess.silo_daily import silo_daily
from DAESIM_preprocess.canopy_height import canopy_height

In [3]:
# Choosing location
lat, lon = -34.3890427, 148.469499

buffer=0.02
stub = "mel-sellheim-1"
start_year = 2020
end_year = 2021

stub="ACT-N-01-2021"
lat=-35.245652
lon=149.094576
buffer=0.1209
start_year = 2021
end_year = 2021

lat, lon = -20.0042, 146.4864
buffer=0.02
stub = "mel-sellheim-1"
start_year = 2020
end_year = 2020

# lat, lon = -34.3890427, 148.469499
# buffer = 0.005  # 0.01 degrees is about 1km in each direction, so 2km total
# stub = "Test"
# start_year = 2020
# end_year = 2021

# Specify output destinations
outdir = os.path.join(gdata_dir, "Data/PadSeg/")
tmp_dir = scratch_dir

In [4]:
%%time
# Download elevation from terrain tiles
terrain_tiles(lat, lon, buffer, outdir, stub, tmp_dir)

terrain tiles filename = /scratch/xe2/tbb801/mel-sellheim-1_terrain_original.tif
Downloaded /scratch/xe2/tbb801/mel-sellheim-1_terrain_original.tif
Saved /g/data/xe2/tbb801/Data/PadSeg/mel-sellheim-1_terrain.tif
CPU times: user 1.36 s, sys: 90.6 ms, total: 1.45 s
Wall time: 8.69 s


In [5]:
%%time
# The CSIRO soils API seems to frequently be randomly unavailable unfortunately
variables = ['Clay', 'Sand', 'Silt', 'pH_CaCl2']
slga_soils(variables, lat, lon, buffer, outdir, stub)

Failed to download Clay
Downloaded /g/data/xe2/tbb801/Data/PadSeg/mel-sellheim-1_Clay.tif
Downloaded /g/data/xe2/tbb801/Data/PadSeg/mel-sellheim-1_Sand.tif
Downloaded /g/data/xe2/tbb801/Data/PadSeg/mel-sellheim-1_Silt.tif
Downloaded /g/data/xe2/tbb801/Data/PadSeg/mel-sellheim-1_pH_CaCl2.tif
CPU times: user 214 ms, sys: 28.5 ms, total: 243 ms
Wall time: 1min 19s


In [6]:
%%time
# Download average climate data from SILO hosted on OzWald
variables = ["Tmax", "Tmin", "Pg"]
ozwald_yearly_average(variables, lat, lon, buffer, start_year, end_year, outdir, stub, tmp_dir)

Saved: /g/data/xe2/tbb801/Data/PadSeg/mel-sellheim-1_Maximum_Temperature_2020_2020_average.tif
Saved: /g/data/xe2/tbb801/Data/PadSeg/mel-sellheim-1_Minimum_Temperature_2020_2020_average.tif
Saved: /g/data/xe2/tbb801/Data/PadSeg/mel-sellheim-1_Annual_Rainfall_2020_2020_average.tif
CPU times: user 178 ms, sys: 88.8 ms, total: 267 ms
Wall time: 2.31 s


#### The 8-day climate modelling data requires access to the Ozwald project on NCI (ub8)

In [12]:
%%time
# Download modelled variables from OzWald
variables = ['Ssoil']
ds = ozwald_8day(variables, lat, lon, buffer, start_year, end_year, outdir, stub)
#def ozwald_8day(variables=["Ssoil", "GPP"], lat=-34.3890427, lon=148.469499, buffer=0.01, start_year="2020", end_year="2021", outdir=scratch_dir, stub="Test"):


ValueError: must supply at least one object to concatenate

###
---


In [7]:
%%time
# Download daily weather data hosted on OzWald
variables = ["VPeff", "Uavg"]
ds = ozwald_daily(variables, lat, lon, buffer, start_year, end_year, outdir, stub, tmp_dir)

Downloaded /scratch/xe2/tbb801/mel-sellheim-1_VPeff_2020.nc
Downloaded /scratch/xe2/tbb801/mel-sellheim-1_Uavg_2020.nc
Saved: /g/data/xe2/tbb801/Data/PadSeg/mel-sellheim-1_ozwald_daily.nc
CPU times: user 257 ms, sys: 107 ms, total: 365 ms
Wall time: 1.72 s


In [8]:
%%time
# Load data pre-downloaded to gdata from SILO's AWS server
ds_silo_daily = silo_daily(["daily_rain", "max_temp", "min_temp", "et_morton_actual", "et_morton_potential"], lat, lon, buffer, start_year, end_year, outdir, stub)
ds_silo_daily

Saved: /g/data/xe2/tbb801/Data/PadSeg/mel-sellheim-1_silo_daily.nc
CPU times: user 149 ms, sys: 1.6 s, total: 1.74 s
Wall time: 6.71 s


<xarray.Dataset>
Dimensions:              (lat: 1, lon: 1, time: 366)
Coordinates:
  * lat                  (lat) float64 -20.0
  * lon                  (lon) float64 146.5
  * time                 (time) datetime64[ns] 2020-01-01 ... 2020-12-31
Data variables:
    daily_rain           (time, lat, lon) float32 0.0 0.0 0.0 ... 15.4 14.8 26.2
    crs                  (time) |S1 b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b''
    max_temp             (time, lat, lon) float32 38.6 36.8 35.4 ... 30.0 30.8
    min_temp             (time, lat, lon) float32 21.1 21.2 21.6 ... 23.1 23.3
    et_morton_actual     (time, lat, lon) float32 4.3 4.9 4.3 ... 5.1 4.4 3.4
    et_morton_potential  (time, lat, lon) float32 10.3 9.4 9.2 ... 7.1 4.8 5.0
Attributes:
    department:               Department of Environment and Science
    department_short:         DES
    copyright:                Copyright - the State of Queensland Department ...
    site_url:                 http://www.longpaddock.qld.gov.au
    institution:              Queensland Government, Department of Environmen...
    raster_source:            Gridded surface was created by interpolating ob...
    raster_source_additions:  and other suppliers (see the SILO webpage for d...
    metadata_url:             http://qldspatial.information.qld.gov.au/catalo...
    reference:                Jeffrey, S.J., Carter, J.O., Moodie, K.B.A. and...
    disclaimer:               1. The user accepts all responsibility and risk...

In [10]:
%%time 
# Download and merge/crop canopy height tiles from their AWS server
canopy_height(lat, lon, buffer, outdir, stub, tmp_dir)

PermissionError: [Errno 13] Permission denied: '/g/data/xe2/datasets/Global_Canopy_Height/311032202.tif.c184fEFD'